In [5]:
!pip install -U transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 138.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct"
)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    torch_dtype="auto",
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [5]:
import torch
from google.colab import drive
import codecs
import pandas as pd

drive.mount('/content/drive')

def create_prompt(text):
  # プロンプトの準備
  chat = [
      { "role": "system", "content": "you are exeellent assistant for natural language processing" },
      { "role": "user", "content": "Followings are the reviews concerning sushi restaurant. Extract only the words representing the reviewers impressions.\
      reviews : {}".format(text) },
  ]
  return chat

path = '/content/drive/My Drive/flavorOAVdataset/'
reviews = pd.read_csv(path + 'tabelog_reviews_en.csv',encoding='ms932', sep=',')

impressions = []
for i,(t,body) in enumerate(zip(reviews['title'],reviews['body'])):


  prompt = tokenizer.apply_chat_template(create_prompt(body), tokenize=False, add_generation_prompt=True)

  # 推論の実行
  token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
  with torch.no_grad():
      output_ids = model.generate(
          token_ids.to(model.device),
          do_sample=True,
          temperature=0.6,
          top_p=0.9,
          max_new_tokens=256,
          eos_token_id=[
              tokenizer.eos_token_id,
              tokenizer.convert_tokens_to_ids("<|eot_id|>")
          ],
      )
  output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
  print(output)
  impressions.append([t]+[body]+[output])


df = pd.DataFrame(impressions,columns=['title','body','impression'])
df.to_csv(path + 'impressions.csv')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Here are the words representing the reviewer's impressions:

* buzz (positive)
* impressed (neutral)
* disappointing (negative)
* relieved (positive)
* delicious (positive)
* satisfying (positive)
* disappointing (negative)
* hesitant (neutral)
* fearing (negative)
* recommend (positive)

Note that some words may have multiple connotations or nuances, but I've tried to extract the primary impression conveyed by each word.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

The words representing the reviewer's impression are:

* "really"
* "enjoyed"

These words convey a positive impression, indicating that the reviewer had a good experience at the sushi restaurant.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Here are the words representing the reviewer's impressions:

* "tears" (expressing sadness or disappointment)
* "discolored" (expressing negative opinion about the appearance of the tuna)
* "completely different" (expressing surprise or disappointment)

These words convey a negative impression of the reviewer's experience at the sushi restaurant.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Here are the words representing the reviewer's impressions:

* Excellent
* Well
* Poorly
* Delicious
* Lacking
* Nice
* Fast
* Hasty
* Nice
* Impressive
* Wonderful
* Good
* Tasty
* Sloppy
* Expected
* Preferred

These words convey the reviewer's positive and negative opinions about the sushi restaurant, including their satisfaction with the food, the skill of the sushi makers, and their overall experience.
The words representing the reviewer's impression are:

* delicious
* best
